# Step1. 관련 패키지 및 모듈 import 하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from torch.nn import init

# Step 2. 와인 데이터셋 구성하기

https://archive.ics.uci.edu/ml/datasets/wine

In [ ]:
colnames=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', 'class'] 
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CNN-main/CNNPytorchCodes/data/wine.csv',names=colnames)
train_data = train_data.sample(frac = 1, random_state = 123)
test_data = train_data.iloc[5000:,:]
test_label = test_data['class']
test_data = test_data.drop(labels='class',axis=1)
# train_label = train_data.iloc[:5000,-1]
train_data = train_data.iloc[:5000,:]
# Generate Test Label
# test_label = pd.DataFrame([0. if item == '0' else 1. for item in test_data['class']])
# test_label = [0. if item == 0 else 1. for item in test_data['class']]

# drop the class infromation
# test_data = test_data.drop(['class'], axis=1)

In [ ]:
test_label.values

array([0, 0, 0, ..., 1, 0, 0])

# Step 3. 다층퍼셉트론 모델 설계하기

In [ ]:
class mymodel(nn.Module):
    def __init__(self, is_train=True):        
        super(mymodel, self).__init__()
        ################################
        ## Problem #1
        self.l1 = torch.nn.Linear(12, 8)
        self.l2 = torch.nn.Linear(8, 4)
        self.l3 = torch.nn.Linear(4, 2)
        ################################
                        
    def forward(self, x):        
        ################################
        ## Problem #1   
        x = torch.sigmoid(self.l1(x))
        x = torch.sigmoid(self.l2(x))
        x = torch.sigmoid(self.l3(x))
        ################################        
        return x

model = mymodel()

# 모델 구조 확인하기

In [ ]:
def print_network(model):    
    def _get_network_description(network):
        '''Get the string and total parameters of the network'''
        if isinstance(network, nn.DataParallel):
            network = network.module
        s = str(network)
        n = sum(map(lambda x: x.numel(), network.parameters()))
        return s, n
    s, n = _get_network_description(model)
    if isinstance(model, nn.DataParallel):
        net_struc_str = '{} - {}'.format(model.__class__.__name__, model.module.__class__.__name__)
    else: net_struc_str = '{}'.format(model.__class__.__name__)
    log = 'Network structure: {}, with parameters: {:,d}'.format(net_struc_str, n)
    return log, s


log, architecture = print_network(model)
print(log)
print(architecture)

Network structure: mymodel, with parameters: 150
mymodel(
  (l1): Linear(in_features=12, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=4, bias=True)
  (l3): Linear(in_features=4, out_features=2, bias=True)
)


# Step 4. Training 함수 구현하기

In [ ]:
def train(train_data, test_data, test_label, batch_size=1000, epochs=1000, learning_rate = 5e-4):
    # training on cpu
    device = torch.device('cpu')
    # random seed
    torch.manual_seed(1)
    # define model
    model = mymodel().to(device)
    # optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0, betas=(0.9, 0.999))        
    # loss fucntion
    criterion = nn.CrossEntropyLoss().to(device)
    
    # now start Training!!!
    running_loss = 0.0
    for epoch in range(epochs):
        model.train()

        # shuffling every epoch        
        train_data_shuffle = train_data
        train_label_shuffle = pd.DataFrame([0. if item == '0' else 1. for item in train_data_shuffle['class']])
        train_label_shuffle = [0. if item == 0 else 1. for item in train_data_shuffle['class']]

        train_data_shuffle = train_data_shuffle.drop(['class'], axis=1)


        for start, end in zip(range(0, len(train_data_shuffle), batch_size), range(batch_size, len(train_data_shuffle), batch_size)):       

            inputs = torch.from_numpy(train_data_shuffle[start:end].values).to(device)
            inputs = inputs.float()
            labels = torch.Tensor(train_label_shuffle[start:end]).to(device)
            labels = labels.long()
                 
            outputs = model(inputs) # 데이터를 모델에 통과하여 결과 예측(순전파)
            optimizer.zero_grad()            
            loss = criterion(outputs, labels)
            
            ######################################################
            ### Problem 2: Backpropagation, Updating Model Weight
            ######################################################
            loss.backward() # 해당 결과를 기반으로 오류 검사
            optimizer.step() # 손실을 바탕으로 역전파 진행 
            # print statistics
            running_loss += loss.item()
        
              
        total = 0
        correct = 0
        if epoch % 50 == 0 or epoch +1 == epochs:
            model.eval()
            with torch.no_grad(): 
                total = len(test_label)
                test_outputs = model(Variable(torch.from_numpy(test_data.values).float()).to(device))               
                ###################################################
                #### Problem 3: Test model every 50 epochs
                ###################################################
                _, predicted = torch.max(test_outputs.data, 1)
                predicted = predicted.numpy()
                correct += (predicted == test_label).sum().item()                             
                print('[Epoch: {}] [Training Loss: {:.6f}] [Accuracy: {:.6f}]'.format(epoch, running_loss / batch_size, correct/total))
                running_loss = 0.0
        model.train()
    print("End Training!")

In [ ]:
train(train_data, test_data, test_label)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[Epoch: 0] [Training Loss: 0.002893] [Accuracy: 0.235805]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[Epoch: 50] [Training Loss: 0.138586] [Accuracy: 0.764195]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[Epoch: 100] [Training Loss: 0.129789] [Accuracy: 0.764195]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[Epoch: 150] [Training Loss: 0.123216] [Accuracy: 0.764195]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[Epoch: 200] [Training Loss: 0.118035] [Accuracy: 0.764195]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[Epoch: 250] [Training Loss: 0.113593] [Accuracy: 0.764195]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[Epoch: 300] [Training Loss: 0.109538] [Accuracy: 0.764195]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[Epoch: 350] [Training Loss: 0.105724] [Accuracy: 0.764195]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[Epoch: 400] [Training Loss: 0.102002] [Accuracy: 0.764195]
<class 'numpy.ndarray'>
<class 